# Introduction


This notebook will analyse the data, focusing on exploring the data and perform data cleaning.
One observation is that, comparing the county level FIPS data with data from https://transition.fcc.gov/oet/info/maps/census/fips/fips.txthttps://transition.fcc.gov/oet/info/maps/census/fips/fips.txt, 
it appears that for all states with state code prefixed with 0, county fips (cfips) are corrupted.
I investigate the issue and fix the values.  
This correction will allow us to correctly associate the codes with the states that they belong to.  
Then, I investigate the spatial and temporal variations of the target feature as well as other features included in the data.  



# Analysis preparation

## Prepare packages

This library is needed for visualization of geographical distributed data.

In [ ]:
# !pip install plotly-geo

Same here.

In [ ]:
# !pip install pyshp

Then, I load the packages used in this EDA.

In [1]:
import pandas as pd
import numpy as np

import plotly.figure_factory as ff
import geopandas
import shapely
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")

## Read data

Train, test nd census data will be explored.

In [2]:
census_df = pd.read_csv('godaddy-microbusiness-density-forecasting/census_starter.csv', low_memory=False)
train_df = pd.read_csv('godaddy-microbusiness-density-forecasting/train.csv', dtype="object")
test_df = pd.read_csv('godaddy-microbusiness-density-forecasting/test.csv', dtype="object")

## Glimpse the data

In [3]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pct_bb_2017            3142 non-null   float64
 1   pct_bb_2018            3142 non-null   float64
 2   pct_bb_2019            3142 non-null   float64
 3   pct_bb_2020            3141 non-null   float64
 4   pct_bb_2021            3141 non-null   float64
 5   cfips                  3142 non-null   int64  
 6   pct_college_2017       3142 non-null   float64
 7   pct_college_2018       3142 non-null   float64
 8   pct_college_2019       3142 non-null   float64
 9   pct_college_2020       3141 non-null   float64
 10  pct_college_2021       3141 non-null   float64
 11  pct_foreign_born_2017  3142 non-null   float64
 12  pct_foreign_born_2018  3142 non-null   float64
 13  pct_foreign_born_2019  3142 non-null   float64
 14  pct_foreign_born_2020  3141 non-null   float64
 15  pct_

There are values for years 2017 to 2021 (5 years) for 5 features:
* pct_bb  
* pct_college  
* pct_foreign_born  
* pct_it_workers  
* median_hh_inc


From the data description, we see that the meaning of these features is as following:

* **pct_bb_[year]** - The percentage of households in the county with access to broadband of any type. Derived from ACS table B28002: PRESENCE AND TYPES OF INTERNET SUBSCRIPTIONS IN HOUSEHOLD.
* **cfips** - The CFIPS code.
* **pct_college_[year]** - The percent of the population in the county over age 25 with a 4-year college degree. Derived from ACS table S1501: EDUCATIONAL ATTAINMENT.
* **pct_foreign_born_[year]** - The percent of the population in the county born outside of the United States. Derived from ACS table DP02: SELECTED SOCIAL CHARACTERISTICS IN THE UNITED STATES.
* **pct_it_workers_[year]** - The percent of the workforce in the county employed in information related industries. Derived from ACS table S2405: INDUSTRY BY OCCUPATION FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER.
* **median_hh_inc_[year]** - The median household income in the county. Derived from ACS table S1901: INCOME IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS).

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   row_id                 122265 non-null  object
 1   cfips                  122265 non-null  object
 2   county                 122265 non-null  object
 3   state                  122265 non-null  object
 4   first_day_of_month     122265 non-null  object
 5   microbusiness_density  122265 non-null  object
 6   active                 122265 non-null  object
dtypes: object(7)
memory usage: 6.5+ MB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25080 entries, 0 to 25079
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              25080 non-null  object
 1   cfips               25080 non-null  object
 2   first_day_of_month  25080 non-null  object
dtypes: object(3)
memory usage: 587.9+ KB


Train data contains, beside the row_id, cfips, county, state as well as first_day_of_month and microbusiness_density and active fields.
Test data contains only row_id, cfips and first_day_of_month.
The value to predict is microbusiness_density for a certain cfips and at a certain first_day_of_month.

In [6]:
train_df.first_day_of_month.unique()

array(['2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01',
       '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01',
       '2020-04-01', '2020-05-01', '2020-06-01', '2020-07-01',
       '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01',
       '2020-12-01', '2021-01-01', '2021-02-01', '2021-03-01',
       '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01',
       '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01',
       '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01',
       '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01',
       '2022-08-01', '2022-09-01', '2022-10-01'], dtype=object)

In [7]:
test_df.first_day_of_month.unique()

array(['2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01',
       '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01'],
      dtype=object)

While train data is from Oct 2019 to Oct 2022, test data is from Nov 2022 to June 2023.

# How many cfips there are?

Let's explore now the cfips data.

In [8]:
census_df.cfips.nunique(), train_df.cfips.nunique(), test_df.cfips.nunique() 

(3142, 3135, 3135)

We have 3135 cfips in train and test data.  
Total number of cfips (in census data) is 3142.  
Not all cfips for which we do have census data are present in train and test datasets.    
Let's see if we do have missing data for certain cfips.   

In [9]:
train_agg = train_df.groupby(["cfips"])["first_day_of_month"].count().reset_index()
train_agg.columns = ["cfips", "dates"]
train_agg.dates.value_counts()

39    3135
Name: dates, dtype: int64

In [10]:
test_agg = test_df.groupby(["cfips"])["first_day_of_month"].count().reset_index()
test_agg.columns = ["cfips", "dates"]
test_agg.dates.value_counts()

8    3135
Name: dates, dtype: int64

Both in train and test data, all cfips have complete number of month, 39 (3 years and 3 months), respectively 8.

# What is the relationship between states, counties and cfips?


Source: https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt

FIPS stands for "Federal Information Processing System" Codes for States and Counties


FIPS codes are numbers which uniquely identify geographic areas.  The number of 
digits in FIPS codes vary depending on the level of geography.  State-level FIPS
codes have two digits, county-level FIPS codes have five digits of which the 
first two are the FIPS code of the state to which the county belongs.  When 
using the list below to look up county FIPS codes, it is advisable to first look
up the FIPS code for the state to which the county belongs.  This will help you
identify the right section of the list while scrolling down, which can be
important since there are over 3000 counties and county-equivalents (e.g.
independent cities, parishes, boroughs) in the United States.

In [11]:
train_df.county.nunique(), train_df.cfips.nunique()

(1871, 3135)

The number of counties is much smaller that the one for FIPS. We assume that only a part of the cfips values corresponds to actual counties and only the cfips associated with real counties, not counties equivalent, are given. Let's make sure all sfips associated with the same state have the same prefix, corresponding to that state.

In [12]:
train_df["cfips_prefix"] = train_df["cfips"].apply(lambda x: str(x)[0:2])
cfips_state = train_df.groupby(["cfips_prefix"])["state"].unique().reset_index()
cfips_state.columns = ["cfips_prefix", "state"]
cfips_state

,cfips_prefix,state
0,10,"[Alabama, Delaware]"
1,11,"[Alabama, District of Columbia]"
2,12,[Florida]
3,13,[Georgia]
4,15,[Hawaii]
5,16,[Idaho]
6,17,[Illinois]
7,18,[Indiana]
8,19,[Iowa]
9,20,"[Alaska, Kansas]"


In [13]:
train_df[["state", "cfips"]].loc[train_df.state.isin(["Alabama", "Arkansas", "Arizona", "California", "Colorado"])].sample(10)

,state,cfips
4208,Arizona,4025
8717,California,6079
7730,California,6029
11093,Colorado,8083
8457,California,6065
807,Alabama,1041
8033,California,6043
9056,California,6097
8305,California,6057
10860,Colorado,8071


It appears that all the cfips for states that starts with 0 are corupted (0 is removed from the prefixes like 01, 02, 03 ...).

We will fix the issue. 

First let's make sure that cfips from the states with FIPS starting with 0 have numbers with only 4 digits.

In [14]:
states_error = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut"]
red_train = train_df.loc[train_df.state.isin(states_error)]
red_train["cfips_len"] = red_train["cfips"].apply(lambda x: len(x))
red_train["cfips_len"].value_counts()

4    12246
Name: cfips_len, dtype: int64

We confiemed that all these cfips are malformed. Let's fix them.


# Fix the malformed cfips



In [15]:
train_df.loc[train_df.state.isin(states_error), "cfips"] = "0" + train_df.loc[train_df.state.isin(states_error), "cfips"]

Let's verify now that they work correctly.

In [16]:
red_train = train_df.loc[train_df.state.isin(states_error)]
red_train["cfips_len"] = red_train["cfips"].apply(lambda x: len(x))
red_train["cfips_len"].value_counts()

5    12246
Name: cfips_len, dtype: int64

And also:

In [17]:
train_df["cfips_prefix"] = train_df["cfips"].apply(lambda x: str(x)[0:2])
cfips_state = train_df.groupby(["cfips_prefix"])["state"].unique().reset_index()
cfips_state.columns = ["cfips_prefix", "state"]
cfips_state

,cfips_prefix,state
0,01,[Alabama]
1,02,[Alaska]
2,04,[Arizona]
3,05,[Arkansas]
4,06,[California]
5,08,[Colorado]
6,09,[Connecticut]
7,10,[Delaware]
8,11,[District of Columbia]
9,12,[Florida]


Everything looks fine now.

# Check the cfips from census data

Let's check that the cfips from census data are all correct as well.


In [18]:
census_df["cfips_len"] = census_df["cfips"].apply(lambda x: len(str(x)))
census_df["cfips_len"].value_counts()

5    2826
4     316
Name: cfips_len, dtype: int64

Let's fix the malformed cfips also here.

In [19]:
census_df["cfips"] = census_df["cfips"].apply(lambda x: str(x))
census_df.loc[census_df.cfips_len==4, "cfips"] = "0" + census_df.loc[census_df.cfips_len==4, "cfips"]

And let's check again now.

In [20]:
census_df["cfips_len"] = census_df["cfips"].apply(lambda x: len(str(x)))
census_df["cfips_len"].value_counts()

5    3142
Name: cfips_len, dtype: int64

# How is microbusiness density distributed in space?


We select microbusiness density value in a certain time moment (from 2021) and represent its distribution on US county-level map.

In [21]:
train_df["microbusiness_density"] = train_df["microbusiness_density"].apply(lambda x: float(x))

In [22]:
selected_data = train_df.loc[train_df.first_day_of_month=="2021-01-01"]
fig = ff.create_choropleth(fips=selected_data.cfips, values=selected_data.microbusiness_density,
                          title="Microbusiness density distribution (county level) for Jan 2021")
fig.layout.template = None
fig.show()

TypeError: 'MultiPolygon' object is not iterable

In [ ]:
selected_data = train_df.loc[train_df.first_day_of_month=="2021-03-01"]
fig = ff.create_choropleth(fips=selected_data.cfips, values=selected_data.microbusiness_density,
                          title="Microbusiness density distribution (county level) for March 2021")
fig.layout.template = None
fig.show()

In [ ]:
selected_data = train_df.loc[train_df.first_day_of_month=="2021-05-01"]
fig = ff.create_choropleth(fips=selected_data.cfips, values=selected_data.microbusiness_density,
                          title="Microbusiness density distribution (county level) for May 2021")
fig.layout.template = None
fig.show()

In [ ]:
selected_data = train_df.loc[train_df.first_day_of_month=="2021-09-01"]
fig = ff.create_choropleth(fips=selected_data.cfips, values=selected_data.microbusiness_density,
                          title="Microbusiness density distribution (county level) for Sept 2021")
fig.layout.template = None
fig.show()

In [ ]:
selected_data = train_df.loc[train_df.first_day_of_month=="2021-12-01"]
fig = ff.create_choropleth(fips=selected_data.cfips, values=selected_data.microbusiness_density,
                          title="Microbusiness density distribution (county level) for Dec 2021")
fig.layout.template = None
fig.show()

# And how census data for same period is distributed?

Let's compare with the percentage of households in the county with access to broadband data distribution for the same year (2021) for which we shown microbusiness density maps. 

In [ ]:
fig = ff.create_choropleth(fips=census_df.cfips, values=census_df.pct_bb_2021,
                          title="Percentage of households in the county with access to broadband (2021)")
fig.layout.template = None
fig.show()

Let's also compare with percent of IT workers from entire working force data distribution from census in the same year.

In [ ]:
fig = ff.create_choropleth(fips=census_df.cfips, values=census_df.pct_it_workers_2021,
                          title="Percentage of IT workers from entire working force (2021)")
fig.layout.template = None
fig.show()

And also compare with the foreign born workers data distribution from census in the same year.

In [ ]:
fig = ff.create_choropleth(fips=census_df.cfips, values=census_df.pct_foreign_born_2021,
                          title="Percentage of foreign born workers (2021)")
fig.layout.template = None
fig.show()

We observe a clear distribution of foreign born people in few areas: California, Texas, Washington state, Florida and New York.


Let's also look to percent of the population in the county over age 25 with a 4-year college degree for the same year.

In [ ]:
fig = ff.create_choropleth(fips=census_df.cfips, values=census_df.pct_college_2021,
                          title="Percent of population in the county over age 25 with a 4-year college degree (2021)")
fig.layout.template = None
fig.show()

We also look to the median household income data for 2021.

In [ ]:
fig = ff.create_choropleth(fips=census_df.cfips, values=census_df.median_hh_inc_2021,
                          title="Median household main income (2021)")
fig.layout.template = None
fig.show()

# How is microbusiness density distributed in time?

Let's now visualise the microbusiness density distribution for FIPS (country-level) for the entire time period in train data.

We will group the counties per state and represent the data variation for few of the states.

In [ ]:
train_mbd = train_df.loc[train_df.state=='Hawaii']
fig = px.line(train_mbd, x="first_day_of_month", y="microbusiness_density", 
              color='county', title="Microbusiness density time variation for Hawaii counties")
fig.show()

In [ ]:
train_mbd = train_df.loc[train_df.state=='District of Columbia']
fig = px.line(train_mbd, x="first_day_of_month", y="microbusiness_density", 
              color='county', title="Microbusiness density time variation for District of Columbia county")
fig.show()

In [ ]:
train_mbd = train_df.loc[train_df.state=='Nevada']
fig = px.line(train_mbd, x="first_day_of_month", y="microbusiness_density", 
              color='county', title="Microbusiness density time variation for Nevada counties")
fig.show()

We will need to check what happens with Carson City and Lincoln County starting from Jan 2021.

In [ ]:
train_mbd = train_df.loc[train_df.state=='Delaware']
fig = px.line(train_mbd, x="first_day_of_month", y="microbusiness_density", 
              color='county', title="Microbusiness density time variation for Delaware counties")
fig.show()

In [ ]:
train_mbd = train_df.loc[train_df.state=='California']
fig = px.line(train_mbd, x="first_day_of_month", y="microbusiness_density", 
              color='county', title="Microbusiness density time variation for California counties")
fig.show()

In [ ]:
train_mbd = train_df.loc[train_df.state=='New York']
fig = px.line(train_mbd, x="first_day_of_month", y="microbusiness_density", 
              color='county', title="Microbusiness density time variation for New York counties")
fig.show()

In [ ]:
train_mbd = train_df.loc[train_df.state=='Texas']
fig = px.line(train_mbd, x="first_day_of_month", y="microbusiness_density", 
              color='county', title="Microbusiness density time variation for Texas counties")
fig.show()

In [ ]:
train_mbd = train_df.loc[train_df.state=='Washington']
fig = px.line(train_mbd, x="first_day_of_month", y="microbusiness_density", 
              color='county', title="Microbusiness density time variation for Washington counties")
fig.show()